In [1]:
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras import utils as np_utils

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_data_folder='Dataset/train/'
test_data_folder='Dataset/test/'
train_files=os.listdir(train_data_folder)

In [3]:
complete_data=None
string_to_int={}
int_to_string={}
counter=0

def fill_data(filename):
    data=np.load(file=train_data_folder+filename)
    filename=filename.split('.')[0]
    temp_var=-1
    
    global counter
    global complete_data
    global string_to_int
    global int_to_string
    
    if filename in string_to_int:
        temp_var=string_to_int[filename]
    
    else:
        temp_var=counter
        counter+=1
        string_to_int[filename]=temp_var
        int_to_string[temp_var]=filename
    
    dimension=data.shape
    temp_y=np.ones(shape=(dimension[0],1),dtype=np.float64)*temp_var
    data=np.concatenate((data,temp_y),axis=1)
    
    if complete_data is None:
        complete_data=np.copy(data)
    
    else:
        complete_data=np.concatenate((complete_data,data),axis=0)
    

for filename in train_files:
    fill_data(filename)

print(complete_data.shape)
print(string_to_int)
print(int_to_string)    

(100000, 785)
{'hat': 4, 'flashlight': 9, 'pig': 15, 'hand': 17, 'penguin': 13, 'spider': 11, 'violin': 7, 'snowman': 10, 'laptop': 18, 'bulldozer': 12, 'skyscraper': 0, 'parrot': 8, 'nose': 1, 'chair': 6, 'trombone': 16, 'eyeglasses': 5, 'banana': 2, 'keyboard': 3, 'harp': 14, 'foot': 19}
{0: 'skyscraper', 1: 'nose', 2: 'banana', 3: 'keyboard', 4: 'hat', 5: 'eyeglasses', 6: 'chair', 7: 'violin', 8: 'parrot', 9: 'flashlight', 10: 'snowman', 11: 'spider', 12: 'bulldozer', 13: 'penguin', 14: 'harp', 15: 'pig', 16: 'trombone', 17: 'hand', 18: 'laptop', 19: 'foot'}


In [4]:
complete_data=np.random.permutation(complete_data)
print(complete_data.shape)

(100000, 785)


In [5]:
[instances,dimensions]=complete_data.shape
x_train=complete_data[:,0:dimensions-1]
y_train=complete_data[:,dimensions-1:].astype(np.int32)
y_train = np_utils.to_categorical(y_train)
print(x_train.shape)
print(y_train.shape)
categories=20
test_x=np.load(file="Dataset/test/test.npy").astype(np.float32)
print(test_x.shape)

(100000, 784)
(100000, 20)
(100000, 784)


In [6]:
# param=1000
# x_train=x_train[0:param,:]
# y_train=y_train[0:param]
# y_train.shape
x_train/=255.0
test_x/=255.0

In [7]:
model = Sequential()
model.add(Dense(output_dim=200, input_dim=784, activation='sigmoid'))
# model.add(Dense(output_dim=40, activation='relu'))
model.add(Dense(output_dim=20, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=200, input_dim=784, activation="sigmoid")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=20, activation="softmax")`
  after removing the cwd from sys.path.


In [12]:
model.fit(x_train, y_train, epochs=50, batch_size=256,validation_split=0.1)
# evaluate the model
scores = model.evaluate(x_train, y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [9]:
def label_to_string(label):
    lis=[]
    instances=label.shape[0]
    for i in range(instances):
        lis.append(int_to_string[label[i]])
    return lis

In [10]:
predictions=np.argmax(model.predict(test_x),axis=1)
print(predictions.shape)
predictions=label_to_string(predictions)
predictions=np.asarray(predictions)
print(predictions.shape)

(100000,)
(100000,)


In [11]:
df=pd.DataFrame(data=predictions,columns=['CATEGORY'])
df.index.name='ID'
df.to_csv(path_or_buf="ignore/NN_sub.csv",sep=',')